In [9]:
import json
import time
from pathlib import Path

import numpy as np
import qibo
from qibo import hamiltonians, set_backend
from boostvqe.models.dbi.double_bracket import (
    DoubleBracketGeneratorType,
    DoubleBracketIteration,
)

from boostvqe.ansatze import VQE, build_circuit
from boostvqe.utils import apply_dbi_steps, rotate_h_with_vqe


In [ ]:

qibo.set_backend("numpy")

# set the path string which define the results
path = "../results/vqe_data/with_params/10q3l/sgd_10q_3l_42/"

# set the target epoch to which apply DBQA and the number of steps
target_epoch = 2000
dbi_steps = 1

# upload system configuration and parameters for all the training
with open(path + "optimization_results.json") as file:
    config = json.load(file)

losses = dict(np.load(path + "energies.npz"))["0"]
params = np.load(path + f"parameters/params_ite{target_epoch}.npy")


# build circuit, hamiltonian and VQE
hamiltonian = hamiltonians.XXZ(nqubits=config["nqubits"], delta=0.5)
circuit = build_circuit(config["nqubits"], config["nlayers"], "numpy")
vqe = VQE(circuit, hamiltonian)
zero_state = hamiltonian.backend.zero_state(config["nqubits"])
zero_state_t = np.transpose([zero_state])
target_energy = np.min(hamiltonian.eigenvalues())


# set target parameters into the VQE
vqe.circuit.set_parameters(params)
vqe_state = vqe.circuit().state()

ene1 = hamiltonian.expectation(vqe_state)


In [2]:
v_vqe = vqe.circuit.unitary()

In [3]:
h_vqe = v_vqe.conj().T @ hamiltonian.matrix @ v_vqe
h_vqe_ham = hamiltonians.Hamiltonian(nqubits = config["nqubits"],matrix=h_vqe)

# This is the scenario that we can try to compile
$D= \sum_i B_i Z_i$
and Group commutator

In [16]:
from qibo import symbols, hamiltonians
b_list = [1+np.sin(x) for x in np.linspace(-1,1,config["nqubits"]) ]
#b_list = np.random.rand(config["nqubits"])
d = hamiltonians.SymbolicHamiltonian( sum([b*symbols.Z(j) for j,b in zip(range(config["nqubits"]),b_list)]))
dm = d.dense.matrix

[Qibo 0.2.9|WARNING|2024-06-07 11:07:33]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


Sam prepared this function to check if the sign is correct. This we should use in eventual hpc training

In [92]:
from copy import deepcopy
dbi = DoubleBracketIteration(
    hamiltonian=deepcopy(h_vqe_ham),
    mode=DoubleBracketGeneratorType.group_commutator_3,
)
import inspect
inspect.getfile(dbi.__class__)

step = 0.12

u_gc3 = dbi.eval_dbr_unitary(step,d=dm, mode = mode)
h_gc3_exposed = hamiltonians.Hamiltonian(matrix = u_gc3.conj().T @ deepcopy(h_vqe_ham).matrix @ u_gc3 )

dbi(0.12,d=dm)
energy_dbi_gc3 = dbi.h.expectation(zero_state_t)
energy_dbi_gc3_exposed = h_gc3_exposed.expectation(zero_state_t)

KeyboardInterrupt: 

In [ ]:
u1 = dbi.h.exp(-step * (np.sqrt(5) - 1) / 2)
u2 = dbi.backend.calculate_matrix_exp(-step * (np.sqrt(5) - 1) / 2, dm)
u3 = dbi.h.exp(step)
u4 = dbi.backend.calculate_matrix_exp(step * (np.sqrt(5) + 1) / 2, dm)
u5 = dbi.h.exp(-step * (3 - np.sqrt(5)) / 2)
u6 = dbi.backend.calculate_matrix_exp(-step, dm)

u_gc3_by_hand = u1 @ u2 @ u3 @ u4 @ u5 @ u6
h_gc3_by_hand = hamiltonians.Hamiltonian(matrix = u_gc3_by_hand.conj().T @ deepcopy(h_vqe_ham).matrix @ u_gc3_by_hand )

energy_dbi_gc3_by_hand = h_gc3_by_hand.expectation(zero_state_t)

In [ ]:
print(ene1,energy_dbi_gc3)

NameError: name 'energy_dbi_gc3' is not defined

# Circuits by hand


In [ ]:
from boostvqe.compiling_XXZ import *
v1_circ = nqubit_XXZ_decomposition(nqubits=config["nqubits"],t=-step * (np.sqrt(5) - 1) / 2,delta=0.5,steps=16)
v1 = v_vqe.conj().T @ v1_circ.unitary() @ v_vqe

v2_circ = nqubit_XXZ_decomposition(nqubits=config["nqubits"],t=step ,delta=0.5,steps=6)
v2 = v_vqe.conj().T @ v2_circ.unitary()@ v_vqe

v3_circ = nqubit_XXZ_decomposition(nqubits=config["nqubits"],t=-step * (3 - np.sqrt(5)) / 2,delta=0.5,steps=16)
v3 = v_vqe.conj().T @ v3_circ.unitary()@ v_vqe

vd1_circ = Circuit(nqubits = config["nqubits"])
vd1_circ.add

In [ ]:
u_gc3_CNOT = v1 @ u2 @ v2  @ u4 @ v3 @ u6


In [23]:
        
for t in np.linspace(.1,.14,5):
    for mode  in [DoubleBracketGeneratorType.group_commutator_3]:        
        print (t)
        dbi(t,d=dm, mode = mode)
        print(dbi.h.expectation(zero_state_t)-ene1)
        dbi.h = deepcopy(h_vqe_ham)

0.1
-0.17370846444809196
0.11000000000000001
-0.1858758485760017
0.12000000000000001
-0.19209367273542632
0.13
-0.19163474209593012
0.14
-0.18406164639815614


-14.751809336151513

In [67]:
_ - ene1

-0.17370846444809196

In [85]:
u_gc3_CNOT = v1 @ u2 @ v2  @ u4 @ v3 @ u6


In [86]:

dbi2 = DoubleBracketIteration(
    hamiltonian=hamiltonians.Hamiltonian(matrix = u_gc3_CNOT.conj().T @ deepcopy(h_vqe_ham).matrix @ u_gc3_CNOT, nqubits = config["nqubits"]),
    mode=DoubleBracketGeneratorType.single_commutator,
)

In [87]:
dbi2.h.expectation(zero_state_t)

-14.74579723717103

In [88]:
_ - ene1

-0.18562264311544752

$\langle 0| V_1\dagger U_\theta^\dagger  H_0 U_\theta V_1 |0\rangle$

$|dbqa \rangle = U_\theta V_1 |0\rangle$

In [19]:
        
for t in np.linspace(.001,.1,3):
    for mode  in [DoubleBracketGeneratorType.single_commutator, DoubleBracketGeneratorType.group_commutator_3]:
        
        print (t)
        dbi(t,d=dm, mode = mode)
        print(dbi.h.expectation(zero_state_t)-ene1)
        dbi.h = deepcopy(h_vqe_ham)

0.001
0.03049416970667984
0.001
-2.93783569489392e-05
0.0505
3.1454674430590615
0.0505
-0.06606296647770904
0.1
7.074245955138497
0.1
-0.17370846444809196


In [25]:
        
for t in np.linspace(.001,.1,13):
    for mode  in [DoubleBracketGeneratorType.single_commutator, DoubleBracketGeneratorType.group_commutator_3]:
        
        print (t)
        dbi(t,d=-dm, mode = mode)
        print(dbi.h.expectation(zero_state_t)-ene1)
        dbi.h = deepcopy(h_vqe_ham)

0.001
-0.02824536086750129
0.001
2.9380340411222505e-05
0.009250000000000001
-0.1680169705742678
0.009250000000000001
0.0025178786739274983
0.0175
-0.12216196003698343
0.0175
0.00904829844227173
0.025750000000000002
0.12697893245887215
0.025750000000000002
0.019708017902930308
0.034
0.5809235629930072
0.034
0.03461163575132886
0.04225
1.2225916652057816
0.04225
0.0538671984955581
0.0505
2.0170847723539627
0.0505
0.07753647566399202
0.058750000000000004
2.9158127725243492
0.058750000000000004
0.1055948106119633
0.067
3.863256246756647
0.067
0.13789621831791443
0.07525000000000001
4.804975983893996
0.07525000000000001
0.1741487964359827
0.0835
5.695149857470556
0.0835
0.2139042537835767
0.09175
6.502038526684503
0.09175
0.2565636208389197
0.1
7.210326149454998
0.1
0.30139922935152264


## We need to schedule unitaries such as this one below

This later needs to be generalized to the GC3 PR from Andrew to get the same performance



In [ ]:
def u_gcr(h,d,t):
    ud =  d.exp(t)
    u = h.exp(-t)
    return     ud.conj().T @ u @ud


In [ ]:
for t in np.linspace(1e-9,1e-6,3):
    u_gcrd = u_gcr(h_vqe_ham,d,t)
    print(h_vqe_ham.expectation(u_gcrd@zero_state_t)-ene1)

5.329070518200751e-15
3.963354089364657e-10
1.5823502508283127e-09


- This doesn't work now. It's better to have the correct expose dbr unitary and then take the exposed unitaries and compare to compiled circuits.


If someone picks it up:
- please try to find the correct scheduling as above
- then expose the unitaries
- and then use the XXZ decomposition function to replace h.exp(-t) by a circuit